<a href="https://colab.research.google.com/github/Lvby/amazon/blob/master/amazonFood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#https://stackoverflow.com/questions/52530578/how-use-tpu-in-google-colab
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

In [0]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/content/drive/My Drive/amazon-fine-food-reviews/')

import attention


In [38]:
import io
import os
import pandas as pd
import numpy as np
#import data that stored on Google drive
import io
df = pd.read_csv('/content/drive/My Drive/amazon-fine-food-reviews/Reviews.csv')
# Dataset is now stored in a Pandas Dataframe
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [39]:
# this block will be needed in the real dataset
df=df.dropna()
df=df[['Summary','Text']].reset_index(drop=True)
df.head()

len(df) #568411

568411

In [41]:
print("preprocessing")
df["Text"]=df["Text"].str.lower()
df["Summary"]=df["Summary"].str.lower()

df.head()

preprocessing


,Summary,Text
0,good quality dog food,i have bought several of the vitality canned d...
1,not as advertised,product arrived labeled as jumbo salted peanut...
2,"""delight"" says it all",this is a confection that has been around a fe...
3,cough medicine,if you are looking for the secret ingredient i...
4,great taffy,great taffy at a great price. there was a wid...


In [42]:
!pip install sentencepiece

In [16]:
import logging
import sentencepiece as spm
import os

logging.basicConfig(level=logging.INFO)

input_file ='to_mytxt2.txt'
vocab_size = 8000
prefix = 'm_bpe'


templates = '--input={} --model_prefix={} --vocab_size={} --pad_id=0 --unk_id=1 --bos_id=2  --eos_id=3 --model_type=bpe'
cmd = templates.format(input_file, prefix, vocab_size)

spm.SentencePieceTrainer.train(cmd)
print("BPE model trained!")


BPE model trained!


In [43]:
# makes segmenter instance and loads the model file (m.model)
sp_bpe = spm.SentencePieceProcessor()
sp_bpe2 = spm.SentencePieceProcessor()
sp_bpe.load('{}.model'.format(prefix))
sp_bpe2.load('{}.model'.format(prefix))
print("Trained BPE model loaded")


#option={"unk": "UNK","eos": "</s>"}
extra_options = 'bos:eos' #'reverse:bos:eos'
sp_bpe.SetEncodeExtraOptions(extra_options)

print("sp_bpe with extra options ")  
print(sp_bpe.EncodeAsIds('so good'))
print(sp_bpe.EncodeAsPieces('so good'))

#print("sp_bpe2 without extra options ")
#print(sp_bpe2.EncodeAsIds('so good'))
#print(sp_bpe2.EncodeAsPieces('so good'))

print(sp_bpe.DecodeIds([5]))
# vocabulary size
print(len(sp_bpe))

Trained BPE model loaded
sp_bpe with extra options 
[2, 110, 143, 3]
['<s>', '▁so', '▁good', '</s>']
a
8000


In [0]:
df.to_csv(r'/content/drive/My Drive/amazon-fine-food-reviews/to_mytxt2.txt', header=None, index=None, sep=' ', mode='a')


In [44]:
import warnings
warnings.filterwarnings("ignore")

df["Tokened_Summary"]=df["Summary"].apply(lambda x:sp_bpe.encode_as_pieces(x))
df["Tokened_Text"]=df["Text"].apply(lambda x:sp_bpe.encode_as_pieces(x))
#val.head(10)


df.head()

,Summary,Text,Tokened_Summary,Tokened_Text
0,good quality dog food,i have bought several of the vitality canned d...,"[<s>, ▁good, ▁quality, ▁dog, ▁food, </s>]","[<s>, ▁i, ▁have, ▁bought, ▁several, ▁of, ▁the,..."
1,not as advertised,product arrived labeled as jumbo salted peanut...,"[<s>, ▁not, ▁as, ▁advertised, </s>]","[<s>, ▁product, ▁arrived, ▁labeled, ▁as, ▁j, u..."
2,"""delight"" says it all",this is a confection that has been around a fe...,"[<s>, ▁"", del, ight, "", ▁says, ▁it, ▁all, </s>]","[<s>, ▁this, ▁is, ▁a, ▁conf, ection, ▁that, ▁h..."
3,cough medicine,if you are looking for the secret ingredient i...,"[<s>, ▁cough, ▁medicine, </s>]","[<s>, ▁if, ▁you, ▁are, ▁looking, ▁for, ▁the, ▁..."
4,great taffy,great taffy at a great price. there was a wid...,"[<s>, ▁great, ▁taffy, </s>]","[<s>, ▁great, ▁taffy, ▁at, ▁a, ▁great, ▁price,..."


In [45]:
df["Tokened_Summary"]=df["Summary"].apply(lambda x:sp_bpe.encode_as_ids(x))
df["Tokened_Text"]=df["Text"].apply(lambda x:sp_bpe.encode_as_ids(x))
#val.head(10)


df.head()

,Summary,Text,Tokened_Summary,Tokened_Text
0,good quality dog food,i have bought several of the vitality canned d...,"[2, 143, 589, 226, 194, 3]","[2, 6, 101, 496, 956, 46, 10, 1310, 545, 1421,..."
1,not as advertised,product arrived labeled as jumbo salted peanut...,"[2, 97, 117, 3030, 3]","[2, 162, 1093, 4698, 117, 123, 187, 960, 4277,..."
2,"""delight"" says it all",this is a confection that has been around a fe...,"[2, 37, 1544, 392, 7969, 1391, 39, 183, 3]","[2, 64, 49, 5, 2464, 2510, 83, 221, 357, 855, ..."
3,cough medicine,if you are looking for the secret ingredient i...,"[2, 5158, 3833, 3]","[2, 202, 77, 103, 774, 62, 10, 4777, 525, 50, ..."
4,great taffy,great taffy at a great price. there was a wid...,"[2, 178, 6268, 3]","[2, 178, 6268, 163, 5, 178, 299, 7968, 312, 10..."


In [46]:
df['summSum']=df['Tokened_Summary'].str.len()
df['summSum'].describe()

count    568411.000000
mean          7.297908
std           3.463988
min           3.000000
25%           5.000000
50%           6.000000
75%           9.000000
max          92.000000
Name: summSum, dtype: float64

In [47]:
df['textSum']=df['Tokened_Text'].str.len()
df['textSum'].describe()

count    568411.000000
mean        105.445635
std         109.038596
min           5.000000
25%          44.000000
50%          72.000000
75%         126.000000
max        5100.000000
Name: textSum, dtype: float64

In [48]:
df.head()

,Summary,Text,Tokened_Summary,Tokened_Text,summSum,textSum
0,good quality dog food,i have bought several of the vitality canned d...,"[2, 143, 589, 226, 194, 3]","[2, 6, 101, 496, 956, 46, 10, 1310, 545, 1421,...",6,56
1,not as advertised,product arrived labeled as jumbo salted peanut...,"[2, 97, 117, 3030, 3]","[2, 162, 1093, 4698, 117, 123, 187, 960, 4277,...",5,44
2,"""delight"" says it all",this is a confection that has been around a fe...,"[2, 37, 1544, 392, 7969, 1391, 39, 183, 3]","[2, 64, 49, 5, 2464, 2510, 83, 221, 357, 855, ...",9,135
3,cough medicine,if you are looking for the secret ingredient i...,"[2, 5158, 3833, 3]","[2, 202, 77, 103, 774, 62, 10, 4777, 525, 50, ...",4,51
4,great taffy,great taffy at a great price. there was a wid...,"[2, 178, 6268, 3]","[2, 178, 6268, 163, 5, 178, 299, 7968, 312, 10...",4,34


In [49]:
#remove columns which length of tokened_summary > 15 and length of tokened_text >60
df = df[(df['textSum'] <60)&(df['summSum']<8)].reset_index(drop=True)
df.head()

,Summary,Text,Tokened_Summary,Tokened_Text,summSum,textSum
0,good quality dog food,i have bought several of the vitality canned d...,"[2, 143, 589, 226, 194, 3]","[2, 6, 101, 496, 956, 46, 10, 1310, 545, 1421,...",6,56
1,not as advertised,product arrived labeled as jumbo salted peanut...,"[2, 97, 117, 3030, 3]","[2, 162, 1093, 4698, 117, 123, 187, 960, 4277,...",5,44
2,cough medicine,if you are looking for the secret ingredient i...,"[2, 5158, 3833, 3]","[2, 202, 77, 103, 774, 62, 10, 4777, 525, 50, ...",4,51
3,great taffy,great taffy at a great price. there was a wid...,"[2, 178, 6268, 3]","[2, 178, 6268, 163, 5, 178, 299, 7968, 312, 10...",4,34
4,"wonderful, tasty taffy",this taffy is so good. it is very soft and ch...,"[2, 810, 7972, 677, 6268, 3]","[2, 64, 6268, 49, 110, 143, 7968, 39, 49, 192,...",6,31


In [0]:
df=df[['Tokened_Text','Tokened_Summary']]
#df.head()

In [51]:
len(df)

171491

In [52]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df, test_size=0.25)
train.shape
#val.shape
train.head()

,Tokened_Text,Tokened_Summary
124528,"[2, 6, 2916, 10, 723, 46, 10, 759, 7968, 6, 11...","[2, 143, 508, 3]"
57429,"[2, 6, 101, 401, 685, 159, 1062, 2971, 2184, 1...","[2, 276, 810, 459, 7977, 3]"
56938,"[2, 64, 105, 5, 178, 858, 7977, 5, 1627, 741, ...","[2, 5, 143, 261, 3]"
100032,"[2, 202, 77, 215, 6366, 7972, 77, 230, 215, 15...","[2, 5174, 3213, 3]"
118550,"[2, 6, 215, 64, 200, 993, 7968, 6, 146, 343, 4...","[2, 200, 62, 491, 3]"


In [53]:
len(train)

128618

In [54]:
val.head()

,Tokened_Text,Tokened_Summary
164243,"[2, 659, 33, 1479, 7972, 1170, 33, 369, 7972, ...","[2, 1170, 30, 659, 3]"
20077,"[2, 335, 2590, 5369, 719, 6, 456, 83, 1986, 46...","[2, 1767, 6502, 482, 3]"
2886,"[2, 6, 155, 2230, 90, 5698, 4450, 7973, 7953, ...","[2, 192, 143, 382, 46, 174, 3]"
170306,"[2, 346, 1447, 746, 163, 1112, 186, 46, 159, 5...","[2, 346, 513, 39, 3]"
74005,"[2, 178, 62, 6860, 7968, 324, 2618, 117, 2802,...","[2, 11, 191, 11, 191, 3]"


In [55]:
len(val)

42873

In [56]:
with open('{}.vocab'.format(prefix), encoding='utf-8') as f:
    vocabs = [doc.strip() for doc in f]

print('num of vocabs = {}'.format(len(vocabs))) # 2000
print(sp_bpe.unk_id())
for id in range (4):
  print(sp_bpe.id_to_piece(id),sp_bpe.is_control(id))

num of vocabs = 8000
1
<pad> True
<unk> False
<s> True
</s> True


In [57]:
x_train=train.iloc[:,0]
x_val=val.iloc[:,0] 
y_train=train.iloc[:,1]
y_val=val.iloc[:,1]
x_train.head()

124528    [2, 6, 2916, 10, 723, 46, 10, 759, 7968, 6, 11...
57429     [2, 6, 101, 401, 685, 159, 1062, 2971, 2184, 1...
56938     [2, 64, 105, 5, 178, 858, 7977, 5, 1627, 741, ...
100032    [2, 202, 77, 215, 6366, 7972, 77, 230, 215, 15...
118550    [2, 6, 215, 64, 200, 993, 7968, 6, 146, 343, 4...
Name: Tokened_Text, dtype: object

In [58]:
from keras.preprocessing.sequence import pad_sequences

#padding zero up to maximum length
x_train   =   pad_sequences(x_train,  maxlen=60, padding='post')
x_val   =   pad_sequences(x_val, maxlen=60, padding='post')
#padding zero up to maximum length
y_train    =   pad_sequences(y_train,  maxlen=8, padding='post')
y_val   =   pad_sequences(x_val, maxlen=8, padding='post')

print(x_train[14])

Using TensorFlow backend.


[   2   39  938   90 5215 1234   30  115  327  183  312 7968   10 2003
  105  143   30   49 5553 7304 7972    6 7948   39   49  897 7968  112
  787   13 2246   27  331 7968   64   49   10  618 2003   15  179   68
  260 1612  581   10  678 3434 7968    6  463    5 2189 7968    3    0
    0    0    0    0]


In [59]:
#we get total unique text tokens x_train =11788
x_voc=0
x_tokens=[]
for row in x_train:
  for tokenx in row:
    if tokenx not in x_tokens:
        x_tokens.append(tokenx)
        #print(token, end=' ')
        x_voc+=1

x_voc=x_voc+1
print(x_voc)


7838


In [60]:
x_voc_size=len(sp_bpe)
print(x_voc_size)

8000


In [61]:
#make a comparision between this one and next cell. 
#we get total unique text tokens y_train =
y_voc=0
y_tokens=[]

for row in y_train:
  for tokeny in row:
    if tokeny not in y_tokens:
      y_tokens.append(tokeny)
     
      y_voc+=1

y_voc=y_voc+1 

print(y_voc)

6081


In [62]:
y_train.shape

(128618, 8)

In [63]:
x_train.shape

(128618, 60)

In [64]:

#we get total unique text tokens y_train =8000 /actually this is the whole voc list size
y_voc2=0
y_tokens2=[]
t = 0
for row in y_train:
  import numpy as np
  t = max(t,np.max(row))
  for token in row:
    if token not in y_tokens2:
      y_tokens2.append(token)
      #print(token,end='')
      y_voc2+=1

y_voc2=y_voc+1 
y_voc2 = t+1

y_voc2_size=len(sp_bpe2)
print(y_voc2_size)


8000


In [0]:
#import csv

#train_csv = train.to_csv (r'/content/drive/My Drive/amazon-fine-food-reviews/train-real.csv', index = None, header=True)
#val_csv = val.to_csv (r'/content/drive/My Drive/amazon-fine-food-reviews/val-real.csv', index = None, header=True)



In [65]:
# i guess the duplate token has too many? i set the vocabulary size to bpe as 8000. May need to change to 15000?
print(len(train))
print(len(val))


128618
42873


In [0]:
#df = pd.read_csv(r'/content/drive/My Drive/amazon-fine-food-reviews/train-real.csv')
##Dataset is now stored in a Pandas Dataframe

#df_v = pd.read_csv(r'/content/drive/My Drive/amazon-fine-food-reviews/val-real.csv')
#len(df_v)

In [66]:
import tensorflow as tf

#from keras.preprocessing.text import Tokenizer 
#from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.15.0


In [0]:
import time
from tensorflow import keras
import datetime, os

#import datetime, os
from tensorflow.python.keras.callbacks import TensorBoard


In [69]:
from keras import backend as K
from tensorflow.keras.layers import *
from attention import *
K.clear_session()

latent_dim = 300
embedding_dim=100  # output vector from embedding layer

#encoder

max_text_len=60

encoder_inputs = Input(shape=(max_text_len,)) 
encoder_embedding = Embedding(x_voc_size, embedding_dim, trainable=True)(encoder_inputs)


#encoder_lstm1
encoder_LSTM1=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output1,state_h1,state_c1=encoder_LSTM1(encoder_embedding)

#encoder_lstm2
encoder_LSTM2=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output2,state_h2,state_c2=encoder_LSTM2(encoder_output1)

#encoder_lstm3
encoder_LSTM3=LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_outputs,state_h,state_c=encoder_LSTM3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc2_size, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc2_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 60, 100)      800000      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 60, 300), (N 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [0]:
#Can I use TensorBoard with Google Colab?
#Tensorboard: ValueError: Duplicate plugins for name projector #22676
#https://github.com/pytorch/pytorch/issues/22676

logs_base_dir = "./logs"
logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=4)

In [0]:
history=model.fit([x_train,y_train[:,:-1]], y_train.reshape(y_train.shape[0],y_train.shape[1], 1)[:,1:] ,epochs=2,callbacks=[es, tensorboard_callback],batch_size=15, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 128618 samples, validate on 42873 samples
Epoch 1/2
  9045/128618 [=>............................] - ETA: 4:23:34 - loss: 2.9757

In [0]:
mytest = sp_bpe.encode_as_ids('love this bbq sauce... can\'t wait for this to be more available, i buy it up if i see a few at the local stores.')
newtest= sp_bpe.encode_as_ids('and I feel like a new man with this bag I looooove it !!')
print(mytest)
test = pad_sequences([newtest],  maxlen=70, padding='post')
print(test)
secNewTest=sp_bpe.DecodeIds(newtest)
#?? means UNK
print(secNewTest)

[2, 215, 64, 2612, 779, 305, 144, 7973, 7949, 1602, 62, 64, 32, 61, 221, 1089, 7972, 6, 260, 39, 253, 203, 6, 374, 5, 581, 167, 10, 771, 917, 7968, 3]
[[   2   30 7947    1  682  127    5  607 1134   90   64  298 7947    1
   101 5749   43   39 7947  400    3    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0]]
and  ⁇  feel like a new man with this bag  ⁇  looooove it !!


In [0]:

# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

max_summary_len = 70
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = 2 #target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = int(np.argmax(output_tokens[0, -1, :]))
        sampled_token = sp_bpe.DecodeIds([sampled_token_index]) #reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!=3):#'eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token_index == 3 or sampled_token_index == 0 or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence
  
#decode_sequence(test)

In [0]:
#define the functions to convert an integer sequence to a word sequence for summary as well as the reviews:

def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_seq[2]) and i!=target_seq[3]):
            newString=newString+sp_bpe.DecodeIds[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+sp_bpe2.DecodeIds[i]+' '
    return newString

In [0]:
!pip install rouge

In [0]:
#test some examples
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to help students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
print(scores)

[{'rouge-1': {'f': 0.5238095189484127, 'p': 0.6285714285714286, 'r': 0.4489795918367347}, 'rouge-2': {'f': 0.2727272680991736, 'p': 0.375, 'r': 0.21428571428571427}, 'rouge-l': {'f': 0.4517704517703128, 'p': 0.5714285714285714, 'r': 0.40816326530612246}}]


In [0]:
for i in range(0,150):
    print("Review:",sp_bpe2.DecodeIds(x_train[i].tolist()))
    print("Original summary:",sp_bpe.DecodeIds(y_train[i].tolist()))
    print("Predicted summary:", decode_sequence(x_train[i].reshape(1,max_text_len)))
 
    #ref. https://pypi.org/project/rouge/
    rouge=Rouge()
    scores = rouge.get_scores(decode_sequence(x_train[i].reshape(1,max_text_len)),sp_bpe.DecodeIds(y_train[i].tolist()),avg=True)
    print(scores)
  
    print("\n")

Review: i gave this as a gift to a man who had everything that he could possibly want. he loved it and i found that he enjoyed the mix very well. thank you for the presentation of this item as well as for the very satisfying results.
Original summary: angel
Predicted summary:  great product 


Review: this is by far my favorite coffee. it goes through my coffee maker without any problems. the crema foam adds a nice touch. this is the only coffee i buy now. it is nice to get this coffee in the usa.
Original summary: my favorite coffee!
Predicted summary:  great coffee 


Review: simple is good. three ingredients. tart and tasty.<br /><br />completely meritorious and not the least bit meretricious.<br /><br />i give them as gifts to dear friends.
Original summary: delicious and nutritious
Predicted summary:  great product 


Review: yesterday my order arrived bur there is on mistake, because nespresso capsules arrived only 1 pack with 10 capsules and i bought 5 packs with 10 capsules. pl

In [0]:
%tensorboard --logdir logs
#click "Graphs" on the menu bar and take a look those generated grpahs

In [0]:

![This image is just show effect of tensorboard.](https://drive.google.com/file/d/1ak1cPo4oTgbYKMOkz0JxBw1QwQ9bJNIf/view?usp=sharing)

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `[This image is just show effect of tensorboard. ](https://drive.google.com/file/d/1ak1cPo4oTgbYKMOkz0JxBw1QwQ9bJNIf/view?usp=sharing)'
